This implamentation is from group 24, composed by:

- Vasco Morganho 81920
- Diogo Moura 86976
- Pedro Pereira 90766


This Notebook showcases the functional part of the first delivery. In each section we  present the function and a set of outputs. After each funtion we will mention the structure and the meaning of each input and output. Alternatively, you can include a standard funtion signature.

Note: This notebook is not functional. Its main goal is to provide instructions on how to call the functions.


In [ ]:
#(a) 

def indexing(D,args = {'topicsFile': 'topics.txt', 'relevanceFile': 'qrels.test', 'preprocessing': False, 'dictFile': 'no-preprocessed-dictionary.comp'}):
    parsingTime = 0
    indexingTime = 0
    memoryAtStart = process.memory_info().rss
    if (args.get('dictFile') != None and args.get('dictFile') in listdir('.')):
        pickleFile = open(args.get('dictFile'), 'rb')
        iIndex = pickle.load(pickleFile)
        pickleFile.close()
        return (iIndex, (0, round(iIndex.getSize()/(1024*1024),2)))

    if(args.get('topicsFile') == None or args.get('relevanceFile') == None or args.get('preprocessing') == None):
        raise ValueError("Missing arguments!\nArguments dictionary must contain 'topicFile', 'relevanceFile' and 'tokenizationType' fields!")

    evaledDocs = getEvaledDocs(args.get('relevanceFile')) 
    filteredD = list(set(filter(lambda x: re.search('/[^/]*?newsML\.xml', x).group()[1:] in evaledDocs ,D)))

    start = time.time()
    iIndex = DocumentInvertedIndex('Train',args.get('topicsFile'), args.get('relevanceFile'), args.get('preprocessing'))
    

    sizeD = len(filteredD)
    for i in tqdm(range(0, sizeD)):
        times = iIndex.addToIndex(filteredD[i])
        parsingTime += times[0]
        indexingTime += times[1]
    
        
    iIndex.calcAllIdfs()
    end = time.time()
    iIndex.calcSize()
    size=round(iIndex.getSize()/(1024*1024),2)#size rounded to 2 decimal places
    totalTime=round(end-start,3)

    if(args.get('dictFile') != None):
        with open(args.get('dictFile'), 'wb') as dictFile:
            pickle.dump(iIndex, dictFile)
    memoryAtEnd = process.memory_info().rss
    print("Parsing Time: {0}s".format(round(parsingTime, 2)))
    print("Indexing Time: {0}s".format(round(indexingTime, 2)))

    return (iIndex, (totalTime, (memoryAtEnd - memoryAtStart) / (1024*1024) ))#returns tuple with (iIndex,(time(s),sizeOccupied(MB)))

__@input:__

__D__ list of strings that corresponds to the path of documents to be indexed.

__args:__ a dictionary with the following keys: topicsFile (file path that contains the topics), 
          relevanceFile (file that contains the relevance feedback), preprocessing (boolean), 
          dictFile (file in binary form from which an inverted index can be retrieved. If the file does not exist
          the inverted index will be stored in the provided file. This argument is optional)

__@output__

A tuple with the folwoing structure (an instance of DocumentInvertedIndex, (total time of parsing, processing and parsing, size of the inverted index))

In [2]:
#(b)

def extract_topic_query(q, I, k,args = {'metric': 'TF-IDF'}):
    topic = None
    q = q.lower()

    if(args.get('metric') == None):
        raise ValueError("You must choose a metric! 'TF-IDF' or 'TF'")

    try:
        topic = I.topicIndex.dictionary[q]
    except KeyError:
        print("Query Topic Not Found")
        return None

    topic = topic[2:]
    
    topicTerms = list(map(lambda x: x[0], topic))
    dictTerms = list(filter(lambda x: x in topicTerms, I.getTerms()))

    if(args.get('metric') == 'TF'):
        # Sum of term frequencies TF 
        paramterizedTerms = [(t, I.getPosting(t)[0]) for t in dictTerms]
        paramterizedTerms = sorted(paramterizedTerms, reverse = True, key = (lambda x: x[1]))[0:k]

    elif(args.get('metric') == 'TF-IDF'):
        #TF*IFDF   TF is in I.getPosting(t)[0] and IDF is in I.getPosting(t)[1] 
        paramterizedTerms = [(t, I.getPosting(t)[0]*I.getPosting(t)[1]) for t in dictTerms]
        paramterizedTerms = sorted(paramterizedTerms, reverse = True, key = (lambda x: x[1]))[0:k]
    else:
        raise ValueError("Invalid metric '{0}' ! Valid Options: 'TF-IDF' or 'TF'".format(args.get('metric')))
    
    return list(map(lambda x: x[0], paramterizedTerms))

__@input:__

__q__ is a string (the topic num, e.g. 'R101')

__k__ is a integer (the size of the extracted query)

__args__: a dictionary with a key: metric(string corresponding to the metric to extract the query from. 'TF-IDF' or 'TF')

__@output__

list of strings

In [3]:
#(c)

def boolean_query(q,I,k,args = {'metric': 'TF-IDF'}):
    query=extract_topic_query(q,I,k, args)
    documents=dict()
    for term in query:
        for doc_id,tf in I.dictionary[term][2:]:#term[0] is the term and term[1] is the tf
            if doc_id not in documents:
                documents[doc_id]=1
            else:
                documents[doc_id]+=1

    documents = [(k, v) for k,v in documents.items()]
    min_term_frequency = math.floor(0.8*len(query))
    documents = sorted(documents, key = lambda x : x[1], reverse = True)
    documents=[doc_id for doc_id,tf in documents if tf >= min_term_frequency]
    return list(filter(lambda x:x in I.topicIndex.getRelevantDocsForTopic(q.lower()) 
                       or x in I.topicIndex.getNonRelevantDocsForTopic(q.lower()),documents))

__@input:__

__q__ is a string (the topic num, e.g. 'R101')

__k__ is a integer

__args__: a dictionary with a key: metric(string corresponding to the metric to extract the query from. Possible values 'TF-IDF' or 'TF'. This is meant to be passed to the extract_topic_query)

__@output__

filtered collection is a ordered list of document identifiers

In [1]:
#(d)

def ranking(q,p,I,args = {'metric': 'TF-IDF'}):
    if(args.get('metric') == 'TF-IDF'):
        similarities = rankingTFIDF(q, p, I)
    elif(args.get('metric') == 'BM25'):
        similarities = rankingBM25(q, p, I)
    elif(args.get('metric') == 'RRF'):
        similarities1 = rankingTFIDF(q, p, I)
        similarities2 = rankingBM25(q, p, I)
        similarities = RRF([similarities1, similarities2])
    else:
        raise ValueError("You must choose a ranking function: 'TF-IDF' or 'BM25' or 'RRF or 'cosSim'")
    similarities = [(k, v) for k,v in similarities.items()]
    similarities = [s for s in similarities if (s[0] in I.topicIndex.getRelevantDocsForTopic(q) or s[0] in I.topicIndex.getNonRelevantDocsForTopic(q))]   
    similarities = sorted(similarities, key= lambda x: x[1], reverse=True)
    similarities=list(filter(lambda x:x[0] in I.topicIndex.getRelevantDocsForTopic(q.lower()) 
                       or x[0] in I.topicIndex.getNonRelevantDocsForTopic(q.lower()),similarities))
    return similarities[0:p]
    
def rankingTFIDF(q,p,I):
    similarities = {}
    for term in I.topicIndex.dictionary[q.lower()]:#iterate over all the terms in the doc query
        tfidf_topic = term[1] #get the topic tf-idf from the dictionary of the topic
        try:
            iIndex=I.dictionary[term[0]] #get the postings from the term in the document Inverted Index
        except KeyError:
            continue
        idf=iIndex[1] #get the idf from the term posting (in the docment Inverted Index)
        for pair in iIndex[2:]:
            docId = pair[0]
            TF_df = pair[1]
            if docId not in similarities.keys():
                similarities[docId] = 0
            similarities[docId] += math.log(1+TF_df,10)* idf * math.log(1+tfidf_topic,10) * idf
    return similarities

def rankingBM25(q, p, I):
    k1 = 1.2
    b = 0.75
    avgDocSum = 0
    for i in I.document_lengths.keys():
        avgDocSum += I.document_lengths[i]
    avgDocLen =  avgDocSum / len(I.document_lengths.keys())
    similarities = { } #idf, #doc_tf
    for term in I.topicIndex.dictionary[q.lower()]:
        postings = I.getPosting(term[0])
        if (postings == None):
            continue
        idfTerm = postings[1]
        for posting in postings[2:]:
            docId = posting[0]
            tf = posting[1]

            if (similarities.get(docId) == None):
                similarities[docId] = 0
            similarities[docId] += idfTerm * ((tf * (k1 + 1)) / (tf + k1 * (1 - b + b * (I.document_lengths[docId]/ avgDocLen))))
    return similarities

def RRF(rankingsList):
    rrf = { }
    for doc in rankingsList[0].keys():
        if (rrf.get(doc) == None):
            rrf[doc] = 0
        for ranking in rankingsList:
            rrf[doc] += (1 / 50 + ranking[doc])
    return rrf

__@input:__

__q__ is a string

__p__ is a integer

__args__: a dictionary with a key: metric(string corresponding to the scoring metric.Possible Values: 'TF-IDF' ,'BM25' or 'RRF')

__@output__

ordered set of top-p documents is a list of pairs (document identifier, scoring)

In [5]:
#(e) 

def evaluation(Q_test,R_test,D_test,args = {'k':10, 'rankingMeasure': 'TF-IDF','booleanMetric': 'TF-IDF', 'queryType': 'ranked'}):
    iIndex=D_test
    queryStartTime = 0
    queryEndTime = 0
    timeSum = 0
    queriesDone = 0
    statsObject = {}
    k=args.get('k')
    if k == None or (not isinstance(k,int)):
        raise ValueError("Invalid k '{0}' !".format(args.get('k')))
    for topic in Q_test:
        statsObject[topic] = {}
        query_result=None
        if (args.get('queryType') != None and args.get('queryType') == 'boolean'):
                queryStartTime = time.time()
                query_result=boolean_query(topic,iIndex,k, {'metric': args.get('booleanMetric')})
                queryEndTime = time.time()
                queriesDone += 1

        elif (args.get('queryType') != None and args.get('queryType') == 'ranked'):
                queryStartTime = time.time()
                query_result = list(map(lambda x: x[0], ranking(topic,k,iIndex, {'metric': args.get('rankingMeasure')})))
                queryEndTime = time.time()
                queriesDone += 1
        else:
            raise ValueError("Invalid query type '{0}' !".format(args.get('queryType')))

        timeSum += queryEndTime - queryStartTime

        if (query_result != None):
            relevantFiles=R_test.getRelevantDocsForTopic(topic)
            nonRelevantFiles = R_test.getNonRelevantDocsForTopic(topic)
            
            recallValues,precisionAtRecall=precisionRecallCurve(query_result, relevantFiles,nonRelevantFiles)
            statsObject[topic]['precision-at-recall'] = precisionAtRecall

            statsObject[topic]['MAP'] = MAP(query_result, relevantFiles, nonRelevantFiles)
            statsObject[topic]['BPREF'] = BPREF(query_result, relevantFiles, nonRelevantFiles)
            statsObject[topic]['Precision'] = calcPrecision(query_result, relevantFiles, nonRelevantFiles)
            statsObject[topic]['Recall'] = calcRecall(query_result, relevantFiles, nonRelevantFiles)
            statsObject[topic]['efficiency'] = queryEndTime - queryStartTime
    return statsObject

__@input:__

__Qtest__ is alist of string containing topic numbers.

__Rtest__ is an instance of TopicIndex (can be accessed through DocumentInvertedIndex.topicIndex)

__Dtest__ is an instance of DocumentInvertedIndex (returned by the indexing function) 

__args__: a dictionary with the following keys/ possible values:
            'k' an integer to pass to the bolean_query as k or to pass to ranking as p;
            'rankingMeasure': 'TF-IDF', 'BM25', 'RRF'. Passed to the ranking function;
            'queryType': 'ranked' or 'boolean' 

__@output__ a dictionary of dictionaries. There's an entry in the first one for each topic in Qtest. Each of these
            entries contains a dictionary with the following entries corresponding to measures: 
            'precision-at-recall', 'MAP', 'BREF', 'Precision' and 'Recall'.


In [6]:
#Further functionalities implemented if thats the case

#text processing options (e.g. absence versus presence of phrase extraction); 
# IR models (including the Boolean, TF-IDF and BM25 models).

__@input:__ and __@output__ of each funtion